In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [6]:
# Check data types
df.dtypes

loan_amnt                     float64
int_rate                      float64
installment                   float64
home_ownership                 object
annual_inc                    float64
                               ...   
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                  object
debt_settlement_flag           object
Length: 86, dtype: object

In [7]:
# Find strings in df
[ col  for col, dt in df.dtypes.items() if dt == object]

['home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'initial_list_status',
 'next_pymnt_d',
 'application_type',
 'hardship_flag',
 'debt_settlement_flag']

In [8]:
# Convert string to numeric values for loan_status
loan_status_dict = {'low_risk': 0, 'high_risk': 1}
df2 = df.replace({'loan_status': loan_status_dict})
#find strings in df
[ col  for col, dt in df2.dtypes.items() if dt == object]

['home_ownership',
 'verification_status',
 'issue_d',
 'pymnt_plan',
 'initial_list_status',
 'next_pymnt_d',
 'application_type',
 'hardship_flag',
 'debt_settlement_flag']

In [9]:
# Convert rest of dataframe from string to numberic values
df2=pd.get_dummies(df2)
df2.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,0,27.24,0.0,0.0,8.0,0.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,0,20.23,0.0,0.0,17.0,1.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,0,24.26,0.0,0.0,8.0,0.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,0,31.44,0.0,1.0,10.0,1.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,0,18.76,0.0,1.0,14.0,0.0,...,1,1,0,1,0,1,1,0,1,1


In [10]:
# Find strings in df2
[ col  for col, dt in df2.dtypes.items() if dt == object]

[]

In [11]:
# Create our features
X = df2.drop("loan_status", axis=1)

# Create the training variables
# converting string values into numerical ones using get_dummies()
X_dummies = pd.get_dummies(X)
X_dummies

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,9.60,0.0,0.0,9.0,0.0,2684.0,...,0,1,0,1,0,1,1,0,1,1
68813,12000.0,0.2727,368.37,63000.0,29.07,0.0,0.0,8.0,0.0,13314.0,...,0,1,0,1,0,1,1,0,1,1
68814,5000.0,0.1992,185.62,52000.0,14.86,0.0,0.0,5.0,1.0,3715.0,...,0,1,0,1,0,1,1,0,1,1
68815,40000.0,0.0646,1225.24,520000.0,9.96,0.0,1.0,21.0,0.0,59529.0,...,0,1,1,0,0,1,1,0,1,1


In [12]:
# Create our target
y = df2['loan_status']
y.unique()

array([0, 1], dtype=int64)

In [13]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [14]:
# Check the balance of our target values
y.value_counts()

0    68470
1      347
Name: loan_status, dtype: int64

In [15]:
# Split train and test group.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(Counter(y_train))
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

Counter({0: 51366, 1: 246})
(51612, 95)
(17205, 95)
(51612,)
(17205,)


In [16]:
# Creating a StandardScaler instance.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [17]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brfc_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# Fitting the model
brfc_model = brfc_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = brfc_model.predict(X_test_scaled)

In [18]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0 (Low Risk)", "Actual 1 (High Risk)"], columns=["Predicted 0 (Low Risk)", "Predicted 1 (High Risk)"]
)
cm_df

,Predicted 0 (Low Risk),Predicted 1 (High Risk)
Actual 0 (Low Risk),15210,1894
Actual 1 (High Risk),36,65


In [19]:
# Calculated the balanced accuracy score
bal_acc_score = balanced_accuracy_score(y_test, predictions)
bal_acc_score

0.7664150126425178

In [20]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.89      0.64      0.94      0.76      0.59     17104
          1       0.03      0.64      0.89      0.06      0.76      0.56       101

avg / total       0.99      0.89      0.65      0.94      0.76      0.59     17205



In [21]:
# Displaying results
print("Balanced Random Forest Classifier")
display(cm_df)
print(f"Accuracy Score : {bal_acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Balanced Random Forest Classifier


,Predicted 0 (Low Risk),Predicted 1 (High Risk)
Actual 0 (Low Risk),15210,1894
Actual 1 (High Risk),36,65


Accuracy Score : 0.7664150126425178
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.89      0.64      0.94      0.76      0.59     17104
          1       0.03      0.64      0.89      0.06      0.76      0.56       101

avg / total       0.99      0.89      0.65      0.94      0.76      0.59     17205



In [22]:
# List the features sorted in descending order by feature importance
features_list = brfc_model.feature_importances_
sorted(zip(brfc_model.feature_importances_, X.columns), reverse=True)

[(0.08200603138527934, 'total_rec_prncp'),
 (0.06396934747748335, 'total_pymnt_inv'),
 (0.05744347415935885, 'total_rec_int'),
 (0.05059089525220024, 'total_pymnt'),
 (0.04957836333074211, 'last_pymnt_amnt'),
 (0.03313864789639978, 'int_rate'),
 (0.02047701317088599, 'issue_d_Jan-2019'),
 (0.0193697462184253, 'installment'),
 (0.017439180890896647, 'out_prncp_inv'),
 (0.016900289617929845, 'tot_hi_cred_lim'),
 (0.016779159090776104, 'dti'),
 (0.015918154332127777, 'mths_since_rcnt_il'),
 (0.015730979750990595, 'annual_inc'),
 (0.015019425852185124, 'total_bc_limit'),
 (0.014632588232213887, 'mo_sin_old_il_acct'),
 (0.014531434631017947, 'avg_cur_bal'),
 (0.014476697543636563, 'mths_since_recent_inq'),
 (0.014297199923957386, 'bc_util'),
 (0.014065599151397613, 'il_util'),
 (0.01372024744521318, 'max_bal_bc'),
 (0.013672087318338818, 'tot_cur_bal'),
 (0.013640477255942788, 'total_rev_hi_lim'),
 (0.01358415277298156, 'loan_amnt'),
 (0.013197281637035508, 'mo_sin_old_rev_tl_op'),
 (0.0131

### Easy Ensemble AdaBoost Classifier

In [23]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)

# Fitting the model
eec_model = eec_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = eec_model.predict(X_test_scaled)

In [24]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df2 = pd.DataFrame(
    cm, index=["Actual 0 (Low Risk)", "Actual 1 (High Risk)"], columns=["Predicted 0 (Low Risk)", "Predicted 1 (High Risk)"]
)
cm_df2

,Predicted 0 (Low Risk),Predicted 1 (High Risk)
Actual 0 (Low Risk),16119,985
Actual 1 (High Risk),8,93


In [25]:
# Calculated the balanced accuracy score
bal_acc_score2 = balanced_accuracy_score(y_test, predictions)
bal_acc_score2

0.931601605553446

In [26]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.94      0.92      0.97      0.93      0.87     17104
          1       0.09      0.92      0.94      0.16      0.93      0.87       101

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



In [27]:
# Displaying results
print("Easy Ensemble Classifier")
display(cm_df2)
print(f"Accuracy Score : {bal_acc_score2}")
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Easy Ensemble Classifier


,Predicted 0 (Low Risk),Predicted 1 (High Risk)
Actual 0 (Low Risk),16119,985
Actual 1 (High Risk),8,93


Accuracy Score : 0.931601605553446
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.94      0.92      0.97      0.93      0.87     17104
          1       0.09      0.92      0.94      0.16      0.93      0.87       101

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205

